In [304]:
import lyricsgenius as genius
from bs4 import BeautifulSoup
import lxml
import requests
import pandas as pd
import apiInfo

In [144]:
# Get top performing hip-hop artists of 2010 decade from wikipedia using BeatifulSoup to scrape table

WIKI_URL = "https://en.wikipedia.org/wiki/List_of_best-selling_hip-hop_albums_of_the_2010s_in_the_United_States"

req = requests.get(WIKI_URL)
soup = BeautifulSoup(req.content, 'html.parser')
table_classes = {"class": ["sortable", "plainrowheaders"]}
table = soup.find("table", { "class" : "wikitable sortable" })

top_songs = pd.DataFrame()

for row in table.find_all("tr"):
    cells = row.find_all("td")
    # some rows have 6 columns (for the year associated with all the albums in the year)
    # this handles the split in the data 
    if len(cells) == 6:
        year = cells[0].find(text=True)
        artist_list = cells[1].findAll(text=True)
        album = cells[2].find(text=True)
        label = cells[3].find(text=True)
        units = cells[4].find(text=True)
        cert = cells[5].find(text=True)
    elif len(cells) == 5:
        artist_list = cells[0].findAll(text=True)
        album = cells[1].find(text=True)
        label = cells[2].find(text=True)
        units = cells[3].find(text=True)
        cert = cells[4].find(text=True)
    if cells:
        for artist in artist_list:
            # hack-y way to handle edge cases for the Genius.com API
            artist = artist.replace("ASAP","A$AP")
            artist = artist.replace("Jay Z","JAY-Z")
            # Take care of multiple artists on list but account for Macklemore & Ryan Lewis
            if "&" not in artist or "Macklemore" in artist:
                #removing some noise, won't translate well to analysis, we can filter any other artists here as well
                if "various artist" not in artist:
                    top_songs = top_songs.append({'year':year,'artist':artist,'album':album,'label':label,'units':units,'cert':cert},ignore_index=True)


In [145]:
# Use a set to get only the unique artists from the list
unique_artists = set(top_songs['artist'])


In [171]:
# Build list of songs using Spotify's API to get top 10 songs of each artist
import spotipy
import spotipy.util as util

token = util.oauth2.SpotifyClientCredentials(client_id=Spotify_client_id, client_secret=Spotify_client_secret)
cache_token = token.get_access_token()
spotify = spotipy.Spotify(cache_token)

spotify_tracks_per_artist = {}

for artist in unique_artists:
    spotify_results = spotify.search(q='artist:' + artist, type='artist')
    # Get the ID of the top search result ... just assume the search returned something good
    spotify_artist_id = spotify_results['artists']['items'][0]['id']
    # Get the top tracks for the artist
    spotify_top_tracks = spotify.artist_top_tracks(spotify_artist_id)
    top_track_list = []
    for track in spotify_top_tracks['tracks']:
        top_track_list.append(track['name'])
    spotify_tracks_per_artist[artist] = top_track_list

#print(spotify_tracks_per_artist)

In [175]:
api = genius.Genius(rapGeniusAPIkey)

In [177]:
songDF = pd.DataFrame()

for ua in unique_artists:
    for song in spotify_tracks_per_artist[ua]:
        lyrics = api.search_song(song,ua)
        songDF = songDF.append({'artist': ua, 'lyrics': lyrics},ignore_index=True)


Searching for "Day 'N' Nite (nightmare)"...
Done.
Searching for "Pursuit Of Happiness (nightmare)"...
Done.
Searching for "Pursuit Of Happiness - Extended Steve Aoki Remix (Explicit)"...
Specified song was not first result :(
Searching for "Erase Me - Main"...
Specified song was not first result :(
Searching for "Just What I Am"...
Done.
Searching for "Soundtrack 2 My Life"...
Done.
Searching for "The Rage"...
Done.
Searching for "Mr. Rager"...
Done.
Searching for "Marijuana"...
Done.
Searching for "Baptized In Fire"...
Done.
Searching for "Still D.R.E."...
Done.
Searching for "The Next Episode"...
Done.
Searching for "Forgot About Dre"...
Done.
Searching for "What's The Difference"...
Done.
Searching for "I Need A Doctor"...
Done.
Searching for "Keep Their Heads Ringin'"...
Done.
Searching for "Xxplosive"...
Done.
Searching for "Kush - Main"...
Done.
Searching for "The Watcher"...
Done.
Searching for "Fuck You"...
Done.
Searching for "Fast Lane"...
Done.
Searching for "Lighters"...
Do

Specified song was not first result :(
Searching for "Kick, Push"...
Done.
Searching for "Wild Child (feat. Jake Torrey)"...
Specified song was not first result :(
Searching for "Little Weapon (feat. Bishop G and Nikki Jean)"...
Specified song was not first result :(
Searching for "Dots & Lines"...
Done.
Searching for "Old School Love (feat. Ed Sheeran)"...
Specified song was not first result :(
Searching for "My Love (feat. Major Lazer, WizKid, Dua Lipa)"...
Specified song was not first result :(
Searching for "Running Back (feat. Lil Wayne)"...
Specified song was not first result :(
Searching for "Lotus Flower Bomb (feat. Miguel)"...
Specified song was not first result :(
Searching for "Ride Out"...
Specified song was not first result :(
Searching for "The Need To Know (feat. SZA)"...
Specified song was not first result :(
Searching for "MY PYT"...
Done.
Searching for "Bad - Remix feat. Rihanna"...
Specified song was not first result :(
Searching for "Ambition (feat. Meek Mill & Rick

Specified song was not first result :(
Searching for "Izzo / In The End"...
Specified song was not first result :(
Searching for "Big Pimpin' / Papercut"...
Specified song was not first result :(
Searching for "H•A•M"...
Specified song was not first result :(
Searching for "Exchange"...
Done.
Searching for "Don't"...
Done.
Searching for "Run Me Dry"...
Done.
Searching for "Right My Wrongs"...
Done.
Searching for "Insecure"...
Specified song was not first result :(
Searching for "Self-Made"...
Done.
Searching for "Sorry Not Sorry"...
Done.
Searching for "Somethin Tells Me"...
Done.
Searching for "Let Me Explain"...
Done.
Searching for "Been That Way"...
Done.
Searching for "Put On"...
Specified song was not first result :(
Searching for "Soul Survivor"...
Specified song was not first result :(
Searching for "American Dream (feat. J. Cole, Kendrick Lamar)"...
Specified song was not first result :(
Searching for "Time (feat. Jeezy)"...
Specified song was not first result :(
Searching for 

In [178]:
# Save the data collected since it takes so long to scrape
songDF.to_pickle("./songDF.pickle")

In [193]:
songDF = pd.read_pickle("./songDF.pickle")

In [195]:
songData = pd.DataFrame()

for index,row in songDF.iterrows():
    song = row['lyrics']
    artist = row['artist']
    if song and hasattr(song,'lyrics') and isinstance(song.lyrics,str):
        words = song.lyrics.lower().replace("\'","").split()
        wordsSet = set(words)
        wordsLength = len(words)
        if wordsLength:
            wordsVariance = len(wordsSet)/wordsLength
            wordsFrequency = {}
            for w in words:
                if w in wordsFrequency:
                    wordsFrequency[w] += 1
                else:
                    wordsFrequency[w] = 1
            songData = songData.append({'artist': artist, 'words': words, 'wordsSet' : wordsSet, 'wordsLength' : wordsLength, 'wordsVariance': wordsVariance, 'wordsFrequency':wordsFrequency}, ignore_index = True)

In [197]:
songData.head()

,artist,words,wordsFrequency,wordsLength,wordsSet,wordsVariance
0,Kid Cudi,"[uh,, uh,, uh,, uh, uh,, uh, day, and, night, ...","{'uh,': 4, 'uh': 2, 'day': 12, 'and': 12, 'nig...",445.0,"{attracting, mind,, cudi-i-i, loner, move, is,...",0.337079
1,Kid Cudi,"[mmmm,, ay-oh, hey..., ratatat,, yeah!, na-na-...","{'mmmm,': 1, 'ay-oh': 1, 'hey...': 1, 'ratatat...",401.0,"{hit, ill, fine, doing, fuck,, thing, screamin...",0.354115
2,Kid Cudi,"[im, just, what, you, made, god, -, not, many,...","{'im': 18, 'just': 15, 'what': 17, 'you': 24, ...",513.0,"{bitches, whatever,, is, go, trust, make, amaz...",0.424951
3,Kid Cudi,"[i, got, 99, problems, and, they, all, bitches...","{'i': 24, 'got': 6, '99': 1, 'problems': 1, 'a...",553.0,"{understand..., bitches, pouring, issues, ones...",0.473779
4,Kid Cudi,"[hands, up,, we, back, in,, born, in, the, rag...","{'hands': 2, 'up,': 1, 'we': 6, 'back': 1, 'in...",573.0,"{grave, mind,, untamed,, same, all,, your, kne...",0.396161


In [206]:
songDF_ALL = pd.DataFrame()

for ua in unique_artists:
    lyrics = api.search_artist(ua,max_songs=100,get_full_song_info=False)
    songDF_ALL = songDF_ALL.append({'artist': ua, 'lyrics': lyrics},ignore_index=True)

Searching for Kid Cudi...

Song 1: "09' Freestyle"
Song 2: "50 Ways to Make a Record"
Song 3: "Adventures"
Song 4: "Afterwards (Bring Yo Friends)"
Song 5: "A Kid Named Cudi [Tracklist + Album Art]"
Song 6: "Alive (Nightmare)"
Song 7: "All Along"
Song 8: "All In"
Song 9: "Amen"
Song 10: "Angels & Demons"
Song 11: "Angered Kids"
Song 12: "Anomaly (Rehearsal Demo)"
Song 13: "Ashin' Kusher"
Song 14: "Balmain Jeans"
Song 15: "Baptized In Fire"
Song 16: "Beez"
Song 17: "Bigger Than You (Do It Alone)"
Song 18: "Brothers"
Song 19: "Brothers (Remix)"
Song 20: "Burn Baby Burn"
Song 21: "By Design"
Song 22: "Call Me Moon Man"
Song 23: "Can I Be"
Song 24: "Can't Look In My Eyes"
Song 25: "Capcom"
Song 26: "Cleveland is the Reason"
Song 27: "Cold Blooded"
Song 28: "Come Around"
Song 29: "CONFUSED!"
Song 30: "Copernicus Landing"
Song 31: "Cosmic Warrior"
Song 32: "Creepers"
Song 33: "Cruisin' (In the Evening)"
Song 34: "CudderIsBack"
Song 35: "Cudi Get"
Song 36: "CuDi Spazzin'"
Song 37: "Cuditastic 

Song 43: "Days Before Rodeo: The Prayer"
Song 44: "Days Before Rodeo [Tracklist + Album Art]"
Song 45: "Dinosaurus"
Song 46: "Don't Play"
Song 47: "Drive"
Song 48: "Drive (The 1975 Remix)"
Song 49: "Drop (Snippet)"
Song 50: "Drugs You Should Try It"
Song 51: "Drunk (Demo)"
Song 52: "Dust"
Song 53: "Easy"
Song 54: "Ecstasy"
Song 55: "Ecstasy (A Remedy)"
Song 56: "Family*"
Song 57: "​first take"
Song 58: "Flexologist"
Song 59: "Flying High"
Song 60: "Forgive Me*"
Song 61: "Gifted (Instrumental)"
Song 62: "God Level"
Song 63: "Gold 88"
Song 64: "​goosebumps"
Song 65: "Goosebumps (Remix ft. Chase atlantic)"
Song 66: "Goosebumps (traduzione ita) (travis verse)"
Song 67: "Green & Purple"
Song 68: "Grey"
Song 69: "​guidance"
Song 70: "Hell of a Night"
Song 71: "High Fashion"
Song 72: "Home"
Song 73: "Hot Sauce"
Song 74: "I Can Tell"
Song 75: "ILLEmeriCA"
Song 76: "Impossible"
Song 77: "Keep Goin' Freestyle"
Song 78: "Kickstart Rollercoaster"
Song 79: "Laptop (Remix)*"
Song 80: "Left Cheek, Ri

Song 45: "Bitch Is Crazy"
Song 46: "Blem"
Song 47: "Blem (Dubplate)"
Song 48: "Bollywood Flow"
Song 49: "Brand New"
Song 50: "Brand New (Remix)"
Song 51: "Bria's Interlude"
Song 52: "Buried Alive (Interlude)"
Song 53: "Cameras"
Song 54: "Cameras / Good Ones Go Interlude - Medley"
Song 55: "Cameras [Official Remix]"
Song 56: "Can I?"
Song 57: "Can't Have Everything"
Song 58: "Can't Hide from Love Freestyle"
Song 59: "Can We Be"
Song 60: "Catch No Feelings"
Song 61: "Cece's Interlude"
Song 62: "Charged Up"
Song 63: "Check it out"
Song 64: "Child's Play"
Song 65: "City is Mine"
Song 66: "Closer"
Song 67: "Closer To My Dreams"
Song 68: "Club Paradise"
Song 69: "Comeback Season"
Song 70: "Come Real"
Song 71: "Come Thru"
Song 72: "Come Winter"
Song 73: "Comments on Macklemore's Post-Grammy Text"
Song 74: "Company"
Song 75: "Congratulations"
Song 76: "Connect"
Song 77: "Controlla"
Song 78: "Controlla (Leak)"
Song 79: "Crew Love"
Song 80: "Dark Thoughts"
Song 81: "Days In The East"
Song 82: "D

Song 52: "G-Eazy Speaks On The Epidemic LP"
Song 53: "G-Eazy's Team"
Song 54: "G-eazy Tim Westwood Freestyle 2017"
Song 55: "Get A Bag"
Song 56: "Get Away (Remix)"
Song 57: "Get Back Up"
Song 58: "Get Mine"
Song 59: "Good For Great"
Song 60: "Good Life"
Song 61: "Good One"
Song 62: "Gotdamn"
Song 63: "Gotta Go"
Song 64: "Groupies"
Song 65: "Grown Up Life"
Song 66: "Hang Ten"
Song 67: "Hang Ten (feat. Erika Flowers)"
Song 68: "Heard 'Em Say (Kanye West Cover)"
Song 69: "Hell Is Round The Corner"
Song 70: "Hello"
Song 71: "Hello (Christoph Andersson Remix)"
Song 72: "Hip-Hop is Cool Again"
Song 73: "Hollywood Freestyle"
Song 74: "I Be In The Lab"
Song 75: "I Don't Believe You"
Song 76: "I Don't Want You Back"
Song 77: "I Mean It"
Song 78: "I Mean It (Biz Remix)"
Song 79: "I Mean It (Remix)"
Song 80: "I Might"
Song 81: "Interlude"
Song 82: "In The Meantime"
Song 83: "Intro"
Song 84: "Intro (The Tipping Point)"
Song 85: "Intro (When It's Dark Out)"
Song 86: "In Your Life"
Song 87: "It's No

Song 35: "Dip Dip (Remix)"
Song 36: "Don't Know What To Do"
Song 37: "Don't Love Me"
Song 38: "Enemies"
Song 39: "Feeling"
Song 40: "Feels Right"
Song 41: "Ferrari Music (Freestyle)"
Song 42: "Fetty Wap - High Thoughts"
Song 43: "Fetty Wap Speaks [1]"
Song 44: "Fetty Wap Speaks [2]"
Song 45: "Fetty Wap [Tracklist + Album Cover]"
Song 46: "Flip Phone"
Song 47: "Flossin' (Freestyle)"
Song 48: "For My Team"
Song 49: "Friends"
Song 50: "Get Back"
Song 51: "Go Hard Boyz"
Song 52: "Go Hard Boyz (Banshee)"
Song 53: "Got A Thang"
Song 54: "Grandma"
Song 55: "Gucci"
Song 56: "Gucci Zoo"
Song 57: "Haha"
Song 58: "Handgun"
Song 59: "Hate You"
Song 60: "Hennessy"
Song 61: "High Thoughts Lyrics"
Song 62: "Hot Niggas (Remy Mix)"
Song 63: "How We Do Things"
Song 64: "I Don't Love Her (Freestyle)"
Song 65: "I'm From Jersey"
Song 66: "I'm Juugin"
Song 67: "I'm Straight"
Song 68: "In My Ways"
Song 69: "Instagram"
Song 70: "Instant Friend"
Song 71: "Into Her"
Song 72: "Island Boy"
Song 73: "Island Boyz"


...

Song 38: "American Gangster [Tracklist + Album Art]"
Song 39: "A Million and One Questions (Remix)"
Song 40: "Analyze This"
Song 41: "Anything"
Song 42: "Anything (2006)"
Song 43: "Appearance on Real Time with Bill Maher"
Song 44: "As One"
Song 45: "A Star Is Born"
Song 46: "A Week Ago"
Song 47: "A Word From Hov"
Song 48: "Backstage freestyle"
Song 49: "Bam"
Song 50: "Barclays Center Freestyle"
Song 51: "BBC"
Song 52: "Beach Chair"
Song 53: "Beach Is Better (Interlude)"
Song 54: "Beanie Sigel Freestlye"
Song 55: "Best of Me (Part 2)"
Song 56: "Best Rapper Alive Mashup"
Song 57: "Beware (Jay-Z Remix)"
Song 58: "Big Chips"
Song 59: "Big Pimpin'"
Song 60: "Big Pimpin' (MTV Unplugged)"
Song 61: "Bitches & Sisters"
Song 62: "BK Anthem"
Song 63: "Black Parade On December 4th"
Song 64: "Blow the Whistle"
Song 65: "Blue Magic"
Song 66: "Blueprint²"
Song 67: "Blueprint (Momma Loves Me)"
Song 68: "Blue's Freestyle/We Family"
Song 69: "Blunts and Armadale (Freestyle)"
Song 70: "Break Up (That's A

Song 42: "Charades"
Song 43: "Check Me Out"
Song 44: "Chicken Marsala"
Song 45: "Chirp"
Song 46: "COLLEGROVE [Album Art + Tracklist]"
Song 47: "Countdown"
Song 48: "Countin"
Song 49: "Cowboy"
Song 50: "Crab Cakes With Mango Salsa"
Song 51: "Crack"
Song 52: "Crack - Album Version (Edited)"
Song 53: "Crib In Closet"
Song 54: "Crib In My Closet"
Song 55: "Daniel Son; Necklace Don [Tracklist + Artwork]"
Song 56: "Day Party"
Song 57: "Days and Days"
Song 58: "Dedication"
Song 59: "Dey Know"
Song 60: "Diamonds Talkin Back"
Song 61: "Dirty Dark"
Song 62: "DJ Bigga Rankin Intro"
Song 63: "Doin Me Daily"
Song 64: "Do My Dance by 2 Chainz"
Song 65: "Don't bring me down"
Song 66: "Don't bring me down - ex amnesia"
Song 67: "Don't Do It"
Song 68: "Don't Do It (Intro)"
Song 69: "Doors Open"
Song 70: "Door Swangin"
Song 71: "Dope Boy"
Song 72: "Dope Peddler"
Song 73: "Dresser (Lil Boy)"
Song 74: "Drug Lord"
Song 75: "Each Erry One Of Em"
Song 76: "El Chapo Jr."
Song 77: "Employee Of The Month"
Song 

Song 66: "Boss Smokers"
Song 67: "Bossy Lady"
Song 68: "Bound 2 Freestyle"
Song 69: "Bout that life"
Song 70: "Box Chevy"
Song 71: "Breathe In, Breathe Out"
Song 72: "Bricks"
Song 73: "Brimstone"
Song 74: "Buried in the streets"
Song 75: "Burn"
Song 76: "Burn (Remix)"
Song 77: "Bury Me a G"
Song 78: "Bust Your Heart"
Song 79: "Buy Back the Block"
Song 80: "Buy Back The Block (Refinance Version)"
Song 81: "Cake (Remix)"
Song 82: "Can't Feel My Face"
Song 83: "Can't Say No"
Song 84: "Career criminal"
Song 85: "Carol City"
Song 86: "Chevy Ridin' High"
Song 87: "Chuck D Speaks / Devil In A New Dress"
Song 88: "Cigar Music"
Song 89: "Clique"
Song 90: "Coke Like the 80's"
Song 91: "Color Money"
Song 92: "Comfortable Freestyle"
Song 93: "Crib in my closet"
Song 94: "Crocodile Python"
Song 95: "Cross That Line"
Song 96: "Cry"
Song 97: "Cut her off"
Song 98: "Cut Her Off (Remix)"
Song 99: "Dead Presidents"
Song 100: "Dead Rappers"

Reached user-specified song limit (100).
Found 100 songs.
Done.

Song 66: "I Come Apart"
Song 67: "In Distress"
Song 68: "I Really Mean It Goblin/ Yonkers Freestyle"
Song 69: "It so Eazy"
Song 70: "I Want (Remix)"
Song 71: "Jacking For Beats Freestyle"
Song 72: "JD"
Song 73: "Jodye"
Song 74: "Jukebox Joints"
Song 75: "Keep It G"
Song 76: "Kissin' Pink"
Song 77: "L.A. Leakers Freestyle"
Song 78: "L.A Leakers Freestyle 2"
Song 79: "L$D"
Song 80: "Leaf (Take 1)"
Song 81: "Leaks Of The Industry (Freestyle)"
Song 82: "Live.Love.A$AP [Mixtape Art]"
Song 83: "Lonesome"
Song 84: "Long.Live.A$AP"
Song 85: "Long.Live.A$AP [Album Art]"
Song 86: "Look"
Song 87: "Lord Pretty Flacko Jodye 2 (LPFJ2)"
Song 88: "Lord Pretty Flacko Joyde 3 (LPFJ III)*"
Song 89: "LVL"
Song 90: "M'$"
Song 91: "Max B"
Song 92: "Max Julien"
Song 93: "Mind Your Own Business"
Song 94: "MONEY BAGS FREESTYLE (DEAN BLUNT MEDITATION)"
Song 95: "M'$ (Single Version)"
Song 96: "Multiply"
Song 97: "Nana Castro Cypher 2010"
Song 98: "New York Bittersweet Symphony"
Song 99: "Niggas In Paris Freesty

Song 66: "Fan In Denial"
Song 67: "Feel Good"
Song 68: "Finding Forever Freestyle"
Song 69: "Fire In The Booth Freestyle"
Song 70: "Flexicution"
Song 71: "Fly Away (Snippet)"
Song 72: "Gang Related"
Song 73: "Gang Related (Commentary Version)"
Song 74: "Get It Done*"
Song 75: "Give It To Me"
Song 76: "Gone"
Song 77: "Grandpa's Space Ship"
Song 78: "Greatest Freestyle of All Time"
Song 79: "Growing Pains"
Song 80: "Growing Pains II"
Song 81: "Growing Pains III"
Song 82: "Growing Pains III (Commentary Version)"
Song 83: "Hallelujah"
Song 84: "Hard Knock TV Freestyle"
Song 85: "Highs & Lows"
Song 86: "Humans"
Song 87: "Hunger"
Song 88: "I Am the Greatest"
Song 89: "I Just Wanna"
Song 90: "​illuminatro"
Song 91: "I Made It"
Song 92: "I'm Gone"
Song 93: "I'm Gone (Commentary Version)"
Song 94: "Incarceration"
Song 95: "Inception"
Song 96: "Indica Badu"
Song 97: "Ink Blot"
Song 98: "In My Mind (Snippet)"
Song 99: "Innermission"
Song 100: "Intro"

Reached user-specified song limit (100).
Foun

In [207]:
#songDF_ALL.to_pickle("./songDF_ALL.pickle")

In [286]:
songData_ALL = pd.DataFrame()

for index,row in songDF_ALL.iterrows():
    for song in row['lyrics'].songs:
        artist = row['artist']
        if song and hasattr(song,'lyrics') and isinstance(song.lyrics,str):
            words = song.lyrics.lower().replace("\'","").split()
            wordsSet = set(words)
            wordsLength = len(words)
            if wordsLength:
                wordsVariance = len(wordsSet)/wordsLength
                wordsFrequency = {}
                for w in words:
                    if w in wordsFrequency:
                        wordsFrequency[w] += 1
                    else:
                        wordsFrequency[w] = 1
                songData_ALL = songData_ALL.append({'artist': artist, 'words': words, 'wordsSet' : wordsSet, 'wordsLength' : wordsLength, 'wordsVariance': wordsVariance, 'wordsFrequency':wordsFrequency}, ignore_index = True)

In [287]:
#songData_ALL.to_pickle("./songData_ALL.pickle")